<a href="https://colab.research.google.com/github/oyeyemixisco/Google_Kaggle_GanAI/blob/main/Day_3_Function_calling_with_the_Gemini_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2024 Google LLC.

In [ ]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Day 3 - Function calling with the Gemini API

Welcome back to the Kaggle 5-day Generative AI course!

In this notebook, you will use the Gemini API's automatic function calling to build a chat interface over a local database. This example is a toy and is missing a number of safety and security constraints you would use in a real-world example.


## Setup

Start by installing and importing the Python SDK.

In [ ]:
%pip install -q -U 'google-generativeai>=0.8.3'

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import google.generativeai as genai

### Set up your API key

To run the following cell, your API key must be stored it in a [Kaggle secret](https://www.kaggle.com/discussions/product-feedback/114053) named `GOOGLE_API_KEY`.

If you don't already have an API key, you can grab one from [AI Studio](https://aistudio.google.com/app/apikey). You can find [detailed instructions in the docs](https://ai.google.dev/gemini-api/docs/api-key).

To make the key available through Kaggle secrets, choose `Secrets` from the `Add-ons` menu and follow the instructions to add your key or enable it for this notebook.

In [ ]:
from kaggle_secrets import UserSecretsClient

GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
genai.configure(api_key=GOOGLE_API_KEY)

If you received an error response along the lines of `No user secrets exist for kernel id ...`, then you need to add your API key via `Add-ons`, `Secrets` **and** enable it.

![Screenshot of the checkbox to enable GOOGLE_API_KEY secret](https://storage.googleapis.com/kaggle-media/Images/5gdai_sc_3.png)

## Create a local database

For this minimal example, you'll create a local SQLite database and add some synthetic data so you have something to query.

Load the `sql` IPython extension so you can interact with the database using magic commands (the `%` instructions) to create a new, empty SQLite database.

In [ ]:
%load_ext sql
%sql sqlite:///sample.db

Create the tables and insert some synthetic data. Feel free to tweak this structure and data.

In [ ]:
%%sql
-- Create the 'products' table
CREATE TABLE IF NOT EXISTS products (
  	product_id INTEGER PRIMARY KEY AUTOINCREMENT,
  	product_name VARCHAR(255) NOT NULL,
  	price DECIMAL(10, 2) NOT NULL
  );

-- Create the 'staff' table
CREATE TABLE IF NOT EXISTS staff (
  	staff_id INTEGER PRIMARY KEY AUTOINCREMENT,
  	first_name VARCHAR(255) NOT NULL,
  	last_name VARCHAR(255) NOT NULL
  );

-- Create the 'orders' table
CREATE TABLE IF NOT EXISTS orders (
  	order_id INTEGER PRIMARY KEY AUTOINCREMENT,
  	customer_name VARCHAR(255) NOT NULL,
  	staff_id INTEGER NOT NULL,
  	product_id INTEGER NOT NULL,
  	FOREIGN KEY (staff_id) REFERENCES staff (staff_id),
  	FOREIGN KEY (product_id) REFERENCES products (product_id)
  );

-- Insert data into the 'products' table
INSERT INTO products (product_name, price) VALUES
  	('MacBook', 799.99),
  	('Keyboard', 129.99),
    ('Hp', 279.99),
  	('ASUS', 429.99),
  	('Mouse', 29.99);

-- Insert data into the 'staff' table
INSERT INTO staff (first_name, last_name) VALUES
  	('Alice', 'Smith'),
  	('Bob', 'Johnson'),
  	('Charlie', 'Williams');

-- Insert data into the 'orders' table
INSERT INTO orders (customer_name, staff_id, product_id) VALUES
  	('David Lee', 1, 1),
  	('Emily Chen', 2, 4),
    ('Dare Azeez', 1, 3),
  	('Emily Chen', 2, 2),
  	('Frank Brown', 1, 5);


 * sqlite:///sample.db
Done.
Done.
Done.
5 rows affected.
3 rows affected.
5 rows affected.


[]

## Define database functions

Function calling with Gemini API's Python SDK can be implemented by defining [an OpenAPI schema](https://ai.google.dev/api/caching#Schema) that is passed to the model. Alternatively you can define Python functions and let the SDK inspect them to automatically define the schema. In this latter case, it's important that the functions are type annotated and have accurate docstrings that describe what the functions do - the model has no insight into the function body, so the docs function as the interface.

By providing three key pieces of functionality - listing tables, describing a table, and executing a query - the LLM (or even another user) will have the basic tools needed to understand and interrogate the database.

Start with a database connection that will be used across all of the functions.

In [ ]:
import sqlite3

db_file = "sample.db"
db_conn = sqlite3.connect(db_file)

The first function will list all tables available in the database. Define it, and test it out to ensure it works.

In [ ]:
def list_tables() -> list[str]:
    """Retrieve the names of all tables in the database."""
    # Include print logging statements so you can see when functions are being called.
    print(' - DB CALL: list_tables')

    cursor = db_conn.cursor()

    # Fetch the table names.
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")

    tables = cursor.fetchall()
    return [t[0] for t in tables]


list_tables()

 - DB CALL: list_tables


['products', 'sqlite_sequence', 'staff', 'orders']

Once the available tables is known, the next step a database user will need is to understand what columns are available in a given table. Define that too, and test that it works as expected.

In [ ]:
def describe_table(table_name: str) -> list[tuple[str, str]]:
    """Look up the table schema.

    Returns:
      List of columns, where each entry is a tuple of (column, type).
    """
    print(' - DB CALL: describe_table')

    cursor = db_conn.cursor()

    cursor.execute(f"PRAGMA table_info({table_name});")

    schema = cursor.fetchall()
    # [column index, column name, column type, ...]
    return [(col[1], col[2]) for col in schema]


describe_table("products")



 - DB CALL: describe_table


[('product_id', 'INTEGER'),
 ('product_name', 'VARCHAR(255)'),
 ('price', 'DECIMAL(10, 2)')]

**The staff Table**

In [ ]:
describe_table("staff")

 - DB CALL: describe_table


[('staff_id', 'INTEGER'),
 ('first_name', 'VARCHAR(255)'),
 ('last_name', 'VARCHAR(255)')]

The order table

In [ ]:
describe_table("orders")

 - DB CALL: describe_table


[('order_id', 'INTEGER'),
 ('customer_name', 'VARCHAR(255)'),
 ('staff_id', 'INTEGER'),
 ('product_id', 'INTEGER')]

Now that the system knows what tables and columns are present, it has enough information to be able to generate and run a `SELECT` query. Now provide that functionality, and test that it works.

In [ ]:
def execute_query(sql: str) -> list[list[str]]:
    """Execute a SELECT statement, returning the results."""
    print(' - DB CALL: execute_query')

    cursor = db_conn.cursor()

    cursor.execute(sql)
    return cursor.fetchall()


execute_query("select * from products")

 - DB CALL: execute_query


[(1, 'MacBook', 799.99),
 (2, 'Keyboard', 129.99),
 (3, 'Hp', 279.99),
 (4, 'ASUS', 429.99),
 (5, 'Mouse', 29.99)]

## Implement function calls

Now you can put it all together in a call to the Gemini API.

Function calling works by adding specific messages to a chat session. When function schemas are defined and made available to the model and a conversation is started, instead of returning a text response, the model may return a `function_call` instead. When this happens, the client must respond with a `function_response`, indicating the result of the call, and the conversation can continue on as normal.

This function calling interaction normally happens manually, allowing you, the client, to validate and initiate the call. However the Python SDK also supports **automatic function calling**, where the supplied functions will be automatically invoked. This is a powerful feature and should only be exposed when it is safe to do so, such as when the functions have no [side-effects](https://en.wikipedia.org/wiki/Side_effect_(computer_science)).

Here's the state diagram representing the conversation flow with function calling. With automatic function calling, the bottom row is executed automatically by the Python SDK. In manual function calling, you write the code to run each step individually.

![function calling state diagram](https://codelabs.developers.google.com/static/codelabs/gemini-function-calling/img/gemini-function-calling-overview_1440.png)

In [ ]:
# These are the Python functions defined above.
db_tools = [list_tables, describe_table, execute_query]

instruction = """You are a helpful chatbot that can interact with an SQL database for a computer
store. You will take the users questions and turn them into SQL queries using the tools
available. Once you have the information you need, you will answer the user's question using
the data returned. Use list_tables to see what tables are present, describe_table to understand
the schema, and execute_query to issue an SQL SELECT query."""

model = genai.GenerativeModel(
    "models/gemini-1.5-pro", tools=db_tools, system_instruction=instruction
)

# Define a retry policy. The model might make multiple consecutive calls automatically
# for a complex query, this ensures the client retries if it hits quota limits.
from google.api_core import retry

retry_policy = {"retry": retry.Retry(predicate=retry.if_transient_error)}

# Start a chat with automatic function calling enabled.
chat = model.start_chat(enable_automatic_function_calling=True)

Now you can engage in a chat conversation where you can ask about the contents of the database.

In [ ]:
resp = chat.send_message("What is the cheapest product?", request_options=retry_policy)
print(resp.text)

 - DB CALL: list_tables
 - DB CALL: describe_table
 - DB CALL: execute_query
The cheapest product is the Mouse.


In [ ]:
resp = chat.send_message("can you draw a chart for the product against prices", request_options=retry_policy)
print(resp)

 - DB CALL: execute_query
response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "| Product Name | Price |\n|-------------|-------|\n| MacBook     | 799.99|\n| Keyboard    | 129.99|\n| Hp           | 279.99|\n| ASUS        | 429.99|\n| Mouse       | 29.99 |\n"
              }
            ],
            "role": "model"
          },
          "finish_reason": "STOP",
          "avg_logprobs": -0.08655436008007496
        }
      ],
      "usage_metadata": {
        "prompt_token_count": 517,
        "candidates_token_count": 77,
        "total_token_count": 594
      }
    }),
)


If you re-use the same [`ChatSession`](https://github.com/google-gemini/generative-ai-python/blob/main/docs/api/google/generativeai/ChatSession.md) object, the conversation will continue statefully. If you wish to start fresh, you can call [`start_chat`](https://github.com/google-gemini/generative-ai-python/blob/main/docs/api/google/generativeai/GenerativeModel.md#start_chat) again, or call [`rewind`](https://github.com/google-gemini/generative-ai-python/blob/main/docs/api/google/generativeai/ChatSession.md#rewind) on the chat object to go back a turn.

Continue the chat here by asking a follow-up question. Note that the database information is preserved, and the context of the specific product is inferred.

In [ ]:
resp = chat.send_message("and how much is it?", request_options=retry_policy)
print(resp.text)

 - DB CALL: execute_query
The cheapest product is Mouse and it costs $29.99. 



Asking the Models different questions

In [ ]:
resp = chat.send_message("who among the staff have the highest sales", request_options=retry_policy)
print(resp.text)

 - DB CALL: list_tables
 - DB CALL: describe_table
 - DB CALL: describe_table
 - DB CALL: execute_query
Alice Smith has the highest sales. 



In [ ]:
resp = chat.send_message("and how much is it?", request_options=retry_policy)
print(resp.text)

 - DB CALL: execute_query
Alice Smith has made $1109.97 in sales. 



In [ ]:
resp = chat.send_message("Is there any customer that bought tow different items", request_options=retry_policy)
print(resp.text)

 - DB CALL: list_tables
 - DB CALL: describe_table
 - DB CALL: execute_query
Yes, Emily Chen bought more than one different item.



In [ ]:
resp = chat.send_message("what did he buy?", request_options=retry_policy)
print(resp.text)

 - DB CALL: execute_query
Emily Chen bought an ASUS and a Keyboard.



Explore the chat session and ask your own questions. If you want to try asking more complex questions, try using the `gemini-1.5-pro` model. It has a lower rate limit, so calls might take a bit longer on the free tier, but you'll notice an improvement in expressivity.

In [ ]:
model = genai.GenerativeModel(
    "models/gemini-1.5-pro-latest", tools=db_tools, system_instruction=instruction
)

chat = model.start_chat(enable_automatic_function_calling=True)
response = chat.send_message('Which salesperson sold the cheapest product?', request_options=retry_policy)
print(response.text)

 - DB CALL: list_tables
 - DB CALL: describe_table
 - DB CALL: describe_table
 - DB CALL: describe_table
 - DB CALL: execute_query
Alice Smith sold the cheapest product.


### Inspecting the conversation

To see the calls that the model makes, and what the client returns in response, you can inspect `chat.history`. This helper function will print out each turn along with the relevant fields passed or returned.

In [ ]:
import textwrap


def print_chat_turns(chat):
    """Prints out each turn in the chat history, including function calls and responses."""
    for event in chat.history:
        print(f"{event.role.capitalize()}:")

        for part in event.parts:
            if txt := part.text:
                print(f'  "{txt}"')
            elif fn := part.function_call:
                args = ", ".join(f"{key}={val}" for key, val in fn.args.items())
                print(f"  Function call: {fn.name}({args})")
            elif resp := part.function_response:
                print("  Function response:")
                print(textwrap.indent(str(resp), "    "))

        print()


print_chat_turns(chat)

User:
  "Which salesperson sold the cheapest product?"

Model:
  Function call: list_tables()

User:
  Function response:
    name: "list_tables"
    response {
      fields {
        key: "result"
        value {
          list_value {
            values {
              string_value: "products"
            }
            values {
              string_value: "sqlite_sequence"
            }
            values {
              string_value: "staff"
            }
            values {
              string_value: "orders"
            }
          }
        }
      }
    }


Model:
  Function call: describe_table(table_name=products)

User:
  Function response:
    name: "describe_table"
    response {
      fields {
        key: "result"
        value {
          list_value {
            values {
              list_value {
                values {
                  string_value: "product_id"
                }
                values {
                  string_value: "INTEGER"
                }


In this output you can see each of the conversational turns that were made. Note that the model doesn't remember anything outside of a `ChatSession` object, so you can make changes to the database structure or data and the model will respond without needing any code changes - try this out!

## Further reading

To learn more about what the Gemini API can do with function calling, check out the [Function calling cookbook](https://github.com/google-gemini/cookbook/blob/main/quickstarts/Function_calling.ipynb) (see `Manual function calling` to understand how function calling works manually) as well as [Function calling config](https://github.com/google-gemini/cookbook/blob/main/quickstarts/Function_calling_config.ipynb), which gives you fine-grained control over how function calling is triggered.